In [1]:
import math
from copy import deepcopy
import numpy as np
import types
import pylab
import matplotlib.pyplot as plt
import plotly.plotly as py
from plotly.graph_objs import Surface
import plotly.tools as tls
import sympy
from sympy import *
tls.set_credentials_file(username='Mungos', api_key='mcnflyescl')

# Nelder Mead

In [2]:
def nelder_mead(f, x_start,
                step=1, no_improve_thr=10e-6,
                no_improv_break=50, max_iter=0,
                alpha=1., gamma=2., rho=-0.5, sigma=0.5,verbose=False):
    
    dim = len(x_start)
    prev_best = f.evaluate(x_start)
    no_improv = 0
    x_start = np.array(x_start).astype(np.float64)
    res = [[x_start, prev_best]]

    for i in range(dim):
        x = deepcopy(x_start)
        x[i] = x[i] + step
        score = f.evaluate(x)
        res.append([x, score])
    
    
    # simplex iter
    iters = 0
    while 1:
        # order
        res.sort(key=lambda x: x[1])
        best = res[0][1]

        # break after max_iter
        if max_iter and iters >= max_iter:
            return res[0][0]
        iters += 1


        if best < prev_best - no_improve_thr:
            no_improv = 0
            prev_best = best
        else:
            no_improv += 1

        if no_improv >= no_improv_break:
            return res[0][0]

        # centroid
        x0 = [0.] * dim
        for tup in res[:-1]:
            for i, c in enumerate(tup[0]):
                x0[i] += c / (len(res)-1)
                
        # break after no_improv_break iterations with no improvement
        if verbose:
            print  'Value in centroid: ', f.evaluate(x0), ' Centroid: ', x0
                
        # reflection
        xr = x0 + alpha*(x0 - res[-1][0])
        rscore = f.evaluate(xr)
        if rscore < res[0][1]:
            xe = x0 + gamma*(x0 - res[-1][0])
            escore = f.evaluate(xe)
            if escore < res[0][1]:
                del res[-1]
                res.append([xe, escore])
                continue
            else:
                del res[-1]
                res.append([xr, rscore])
                continue
        else:
            if rscore > res[-2][1]:
                if rscore < res[-1][1]:
                    del res[-1]
                    res.append([xr, rscore])
                # contraction
                xc = x0 + rho*(x0 - res[-1][0])
                cscore = f.evaluate(xc)
                if cscore < res[-1][1]:
                    del res[-1]
                    res.append([xc, cscore])
                    continue
                else:
                    # reduction
                    x1 = res[0][0]
                    nres = []
                    for tup in res:
                        redx = x1 + sigma*(tup[0] - x1)
                        score = f.evaluate(redx)
                        nres.append([redx, score])
                    res = nres
            else:
                del res[-1]
                res.append([xr, rscore])


# if __name__ == "__main__":
    
# #     fun = Function(f4)
#     print nelder_mead(Rosenbrock(), [5,5])

# Golden section

In [3]:
def golden_section_search(f,starting_point = None, a=None,b=None,eps=1, verbose = False):
    if starting_point != None:
        a,b = find_unimodal(starting_point,f.evaluate,eps)
    elif a==None or b== None:
        raise Exception("Starting point or unimodal interval needs to be given")
    
    if verbose:
        print "Searching in interval [%f, %f]" % (a,b)
    
    
    fi = (math.sqrt(5.0) - 1.0)/2
    c = b - (b - a)*fi
    d = a + (b - a)*fi
    while (b - a) > eps:
        c_score, d_score = f.evaluate(c),f.evaluate(d)
        if verbose:
            print "|a = %.3f|c = %.3f|d = %.3f|b = %.3f|f(c) = %.3f|f(d) = %.3f|f(c) > f(d) = %s" %(a,c,d,b,c_score,d_score, c_score>d_score)
        if c_score >= d_score:
            a = c
            c = d
            d = a + (b - a)*fi
        else:
            b = d
            d = c
            c = b - (b - a)*fi
    if verbose:
        print ''
        print "Final interval = [",a,", ", b,"]"
    return (a+b)/2.

# Unimodalni interval

In [4]:
def find_unimodal(starting_point, function, starting_step=1):
    step = starting_step
    current_point = starting_point
    direction = 1
    if function(current_point + step) >= function(current_point):
        direction=-1
    next_point = current_point + direction*step
    cnt = 1
    previous_point = current_point
    while function(current_point) > function(next_point):
        previous_point = current_point
        current_point = next_point
        next_point = starting_point + direction * (2**cnt) * step
        cnt+=1
    
    if previous_point > next_point:
        return next_point, previous_point
    return previous_point, next_point

# Base function class

In [29]:
class Function():
    def subs_dict(self,x):
        dic = {}
        for i,var in enumerate(self.variables):
            dic[var] = x[i]
        return dic
    
    def gradient_symbolic(self):
        grads = []
        for var in self.variables:
            dif = diff(self.equation,var)
            grads.append(dif)
        return grads
        
    def gradient(self,x):
        self.gradient_calls+=1
        sub_dict = self.subs_dict(x)
        grads = self.gradient_symbolic()
        for i,grad in enumerate(grads):
            grads[i]=grad.evalf(subs=sub_dict)
        return np.array(grads,dtype=np.float32)
        
    def hessean_symbolic(self):
        grads = self.gradient_symbolic()
        hessean = []
        for i,grad in enumerate(grads):
            hess_row = []
            for var in self.variables:
                dif = diff(grad,var)
                hess_row.append(dif)
            hessean.append(hess_row)
        return hessean
        
    def hessean(self,x):
        self.hessean_calls+=1
        hess = self.hessean_symbolic()
        sub_dict = self.subs_dict(x)
        for i,row in enumerate(hess):
            for j, val in enumerate(row):
                hess[i][j] = hess[i][j].evalf(subs=sub_dict)
        return np.array(hess,dtype=np.float32)
        
    def evaluate(self,x, exponential=False):
        self.calls+=1
        subs = self.subs_dict(x)
        value = np.array(self.equation.evalf(subs=subs),dtype=np.float32)
        return value
    

# Helper extensions

In [144]:
def is_numeric(x):
    NumberTypes = (types.IntType, types.LongType, types.FloatType)
    return isinstance(x, NumberTypes)

class BoxOptFunction():
    def __init__(self, func,inequality_restriction):
        self.calls = 0
        self.gradient_calls=0
        self.hessean_calls=0
        self.inequality_restriction = inequality_restriction
        self.func = func
        
    def check_inequality_constraints(self,x):
        for constraint in self.inequality_restriction:
            if(constraint.evaluate(x)<0):
                return False
        return True
            

    def evaluate(self,X):
        self.calls+=1
        return self.func.evaluate(X)
    

class MixedTransformationFunction():
    def __init__(self, func,inequality_restriction=None, equality_restriction=None):
        self.calls = 0
        self.gradient_calls=0
        self.hessean_calls=0
        self.t = 1
        self.inequality_restriction = inequality_restriction
        self.equality_restriction = equality_restriction
        self.func = func
        
    def combined_symbolic_grad(self): 
        func_grads = self.func.gradient_symbolic()
        if self.inequality_restriction:
            for eq in self.inequality_restriction:
                for i in range(len(func_grads)):
                    func_grads[i] -= (1./self.t*eq.gradient_symbolic())[i]
                
        if self.equality_restriction:
            for eq in self.equality_restriction:
                for i in range(len(func_grads)):
                    func_grads[i] +=self.t*eq.gradient_symbolic()[i]
        return func_grads
    
    def combined_symbolic_hessean(self):
        grads = self.combined_symbolic_grad()
        hessean = []
        for i,grad in enumerate(grads):
            hess_row = []
            for var in self.func.variables:
                dif = diff(grad,var)
                hess_row.append(dif)
            hessean.append(hess_row)
        return hessean
    
    def combined_grad(self,x):
        self.gradient_calls+=1
        sub_dict = self.subs_dict(x)
        grads = self.combined_symbolic_grad()
        for i,grad in enumerate(grads):
            grads[i]=grad.evalf(subs=sub_dict)
        return np.array(grads,dtype=np.float32)
    
    def hessean(self,x):
        self.hessean_calls+=1
        hess = self.combined_symbolic_hessean()
        sub_dict = self.subs_dict(x)
        for i,row in enumerate(hess):
            for j, val in enumerate(row):
                hess[i][j] = hess[i][j].evalf(subs=sub_dict)
        return np.array(hess,dtype=np.float32)
        
    def set_t(self,t):
        self.t = t
        
    def evaluate(self,X):
        self.calls+=1
        func_value = self.func.evaluate(X)
        if self.inequality_restriction:
            constraint_sum = 0
            for eq in self.inequality_restriction:
                try:
                    val = eq.evaluate(X)
                except TypeError:
                    return 1e10
                constraint_sum +=val
            func_value -= 1./self.t*constraint_sum
                
        if self.equality_restriction:
            constraint_sum=0
            for eq in self.equality_restriction:
                val = eq.evaluate(X)
                constraint_sum +=val
            func_value += self.t*constraint_sum
        return func_value

# Rosenbrock

In [31]:
class Rosenbrock(Function):
    def __init__(self):
        self.calls=0
        self.gradient_calls=0
        self.hessean_calls=0
        self.x1= symbols('x1')
        self.x2 = symbols('x2')
        self.variables = [self.x1,self.x2]
        self.equation = (100*(self.x2-self.x1**2)**2+(1-self.x1)**2)

# F2

In [32]:
class F2(Function):
    def __init__(self):
        self.calls=0
        self.gradient_calls=0
        self.hessean_calls=0
        self.x1= symbols('x1')
        self.x2 = symbols('x2')
        self.variables = [self.x1,self.x2]
        self.equation = ((self.x1-4.)**2 + 4.*(self.x2-2)**2)

# F3

In [33]:
class F3(Function):
    def __init__(self):
        self.calls=0
        self.gradient_calls=0
        self.hessean_calls=0
        self.x1= symbols('x1')
        self.x2 = symbols('x2')
        self.variables = [self.x1,self.x2]
        self.equation = ((self.x1-2.)**2 + (self.x2+3)**2)

# F4

In [34]:
class F4(Function):
    def __init__(self):
        self.calls=0
        self.gradient_calls=0
        self.hessean_calls=0
        self.x1= symbols('x1')
        self.x2 = symbols('x2')
        self.variables = [self.x1,self.x2]
        self.equation = ((self.x1-3.)**2 + (self.x2)**2)

# Restrictions 

In [74]:
class First_restriction(Function): #x2-x1 >= 0
    def __init__(self, inequality_rest=True,power=2):
        self.calls=0
        self.gradient_calls=0
        self.hessean_calls=0
        self.x1= symbols('x1')
        self.x2 = symbols('x2')
        self.variables = [self.x1,self.x2]
        if inequality_rest:
            self.equation = log(self.x2-self.x1)
        else:
            self.equation = (self.x2-self.x1)**power
        
class Second_Restriction(Function): #x2-x1 >= 0
    def __init__(self, inequality_rest=True,power=2):
        self.calls=0
        self.gradient_calls=0
        self.hessean_calls=0
        self.x1= symbols('x1')
        self.x2 = symbols('x2')
        self.variables = [self.x1,self.x2]
        if inequality_rest:
            self.equation = log((2-self.x1))
        else:
            self.equation = (2-self.x1)**power
       
    
class Third_Restriction(Function): #3-x1-x2 >= 0
    def __init__(self, inequality_rest=True,power=2):
        self.calls=0
        self.gradient_calls=0
        self.hessean_calls=0
        self.x1= symbols('x1')
        self.x2 = symbols('x2')
        self.variables = [self.x1,self.x2]
        if inequality_rest:
            self.equation = log((3-self.x1-self.x2))
        else:
            self.equation = (3-self.x1-self.x2)**power
            
            
class Fourth_Restriction(Function): #3+1.5*x1-x2 >= 0
    def __init__(self, inequality_rest=True,power=2):
        self.calls=0
        self.gradient_calls=0
        self.hessean_calls=0
        self.x1= symbols('x1')
        self.x2 = symbols('x2')
        self.variables = [self.x1,self.x2]
        if inequality_rest:
            self.equation = log((3+1.5*self.x1-self.x2))
        else:
            self.equation = (3+1.5*self.x1-self.x2)**power
            
class Fifth_Restriction(Function): #3-x1-x2 >= 0
    def __init__(self, inequality_rest=True,power=2):
        self.calls=0
        self.gradient_calls=0
        self.hessean_calls=0
        self.x1= symbols('x1')
        self.x2 = symbols('x2')
        self.variables = [self.x1,self.x2]
        if inequality_rest:
            self.equation = log((self.x2-1))
        else:
            self.equation = (self.x2-1)**power



# Trazenje unutarnje tocke

In [36]:
class Inner_Point_Search():
    def __init__(self, nejednadzbe,t=None):
        self.nejed = nejednadzbe
        if t:
            self.t = t
        else:
            self.t = 1
    
    def evaluate(self,x):
        total_val = 0
        for nejed in self.nejed:
            value = nejed.evaluate(x)
            if value < 0:
                total_val -= value * self.t
        return total_val
    
    def get_inner_point(self,dimensions,iters=30):
        point = np.random.randint(-10,10,size=dimensions)
        for i in range(iters):
            point = nelder_mead(self, point)
            if evaluate(point) == 0:
                return point
            self.t *=2
        return point
    
# nejed = [F2()]
# inner = Inner_Point_Search(nejed,5)
# print inner.get_inner_point(2)

# Gradijentni spust

In [45]:
def grad_descent(fun, starting_point, epsilon = 1e-6, use_golden_section=True, patience = 50, verbose = False, grad_norm = 1e-2):
    norma = np.linalg.norm(fun.gradient(starting_point))
    current_point = np.array(starting_point).copy()
    if norma < epsilon:
        print "Euclidean norm %s less than od %s stopping search" % (norma, epsilon)
        return current_point
    
    best_sol, best_sol_value = current_point.copy(), fun.evaluate(current_point)
    current_patience = 0
    while True:
        gradient = fun.gradient(current_point)
        norma = np.linalg.norm(gradient)
        if verbose:
            print "Current best point %s with value %s" % (best_sol,best_sol_value)
        if norma < grad_norm:
            print "Euclidean norm %s less than od %s stopping search" % (norma, grad_norm)
            return best_sol
        
        
        if use_golden_section:
            gradient_normed = gradient/np.linalg.norm(gradient)
            
            def function_1D_wrapper(function, array,grads):
                cpy = array.copy()
                class Decorator:
                    def __init__(self,cpy,grads,function):
                        self.grads = grads
                        self.cpy = cpy
                        self.function = function

                    def evaluate(self,x):
                        val = self.cpy - x * self.grads
                        return self.function.evaluate(val)
                return Decorator(cpy,grads,function)
            
            optimal_lambda = golden_section_search(function_1D_wrapper(fun,current_point,gradient_normed),1,eps=1e-6)
            current_point -= optimal_lambda*gradient_normed
        else:
            current_point-= gradient
            
        if np.isnan(current_point).any() or (current_point==np.inf).any():
            print "Divergence detected, stopping search with best found solution %s" %(best_sol)
            return best_sol
        
        if best_sol_value <= fun.evaluate(current_point):
            current_patience +=1
            if(current_patience > patience):
                print "No improvement, stopping search with best found solution %s" %(best_sol)
                return best_sol
        else:
            current_patience = 0
            best_sol, best_sol_value = current_point.copy(), fun.evaluate(current_point)
    
            
#grad_descent(F2(),[3.,3.])

# Newton Rhapson

In [127]:
def newton_rhapson(fun, starting_point, epsilon = 1e-6, use_golden_section=True, patience = 50, verbose = False, grad_norm = 1e-2):
    gradient = fun.gradient(starting_point).dot(np.linalg.inv(fun.hessean(starting_point)))
    norma = np.linalg.norm(gradient)
    current_point = np.array(starting_point, dtype=np.float32).copy()
    if norma < epsilon:
        print "Euclidean norm %s less than od %s stopping search" % (norma, epsilon)
        return current_point
    
    best_sol, best_sol_value = current_point.copy(), fun.evaluate(current_point)
    current_patience = 0
    while True:
        gradient = fun.gradient(current_point).dot(np.linalg.inv(fun.hessean(current_point)))
        norma = np.linalg.norm(gradient)
        if verbose:
            print "Current best point %s with value %s" % (best_sol,best_sol_value)
        if norma < grad_norm:
            print "Euclidean norm %s less than od %s stopping search" % (norma, grad_norm)
            return current_point
        
        
        if use_golden_section:
            gradient_normed = gradient/np.linalg.norm(gradient)
            
            def function_1D_wrapper(function, array,grads):
                cpy = array.copy()
                class Decorator:
                    def __init__(self,cpy,grads,function):
                        self.grads = grads
                        self.cpy = cpy
                        self.function = function

                    def evaluate(self,x):
                        val = self.cpy - x * self.grads
                        return self.function.evaluate(val)
                return Decorator(cpy,grads,function)
            
            optimal_lambda = golden_section_search(function_1D_wrapper(fun,current_point,gradient_normed),1,eps=1e-6)
            current_point -= optimal_lambda*gradient_normed
        else:
            current_point-= gradient
            
        if np.isnan(current_point).any() or (current_point==np.inf).any():
            print "Divergence detected, stopping search with best found solution %s" %(best_sol)
            return best_sol
        
        if best_sol_value <= fun.evaluate(current_point):
            current_patience +=1
            if(current_patience > patience):
                print "No improvement, stopping search with best found solution %s" %(best_sol)
                return best_sol
        else:
            current_patience = 0
            best_sol, best_sol_value = current_point.copy(), fun.evaluate(current_point)
    
            
#newton_rhapson(F2(),[3.,3.])

# Box

In [52]:
def check_explicit_valid(explicit_restrictions,current_point, per_variable_restriction):
    if per_variable_restriction:
        for i,point in enumerate(current_point):
            if i < len(explicit_restrictions):
                if not (explicit_restrictions[i][0] < point < explicit_restrictions[i][1]):
                    return False
    else:
        for point in current_point:
            if not (explicit_restrictions[0] < point < explicit_restrictions[1]):
                return False
    return True
            
def calculate_centroid(worst_point_index, points):
    centroid = np.zeros(points[0].shape)
    for i,point in enumerate(points):
        if i != worst_point_index:
            centroid += point
    return centroid/(len(points)-1)

def move_to_centroid(point,current_point):
    new_point = 1./2 * (point + current_point)
    return new_point

def stopping_condition(centroid_score,scores, epsilon):
    diff = np.sum(np.array(scores)-centroid_score)
    return np.sqrt(diff/len(scores)) <= epsilon
        
def box_opt(fun, starting_point, explicit_restrictions, epsilon = 1e-6, alpha = 1.3, patience=50):
    current_point = np.array(starting_point, dtype=np.float32).copy()
    dimensions = current_point.shape[0]
    per_variable_restriction = True
    if(len(np.array(explicit_restrictions).shape) == 1):
        per_variable_restriction = False
    
    if not (check_explicit_valid(explicit_restrictions, current_point, per_variable_restriction) 
                    or fun.check_inequality_constraints(current_point)):
        raise Exception("Argument needs to be within the restricted area")
    
    points = []
    for i in range(0,2*dimensions+1):
        if per_variable_restriction:
            point = []
            for dim in range(dimensions):
                if dim < len(explicit_restrictions):
                    point.append(np.random.uniform(explicit_restrictions[dim][0],explicit_restrictions[dim][1],1)[0])
                else:
                    point.append(np.random.uniform(-1000,1000))
                    
            point = np.array(point) 
            while fun.check_inequality_constraints(point) == False:
                point = move_to_centroid(point,current_point)
            points.append(point)
            
        else:
            point = np.random.uniform(explicit_restrictions[0],explicit_restrictions[1],dimensions)
            while fun.check_inequality_constraints(point) == False:
                point = move_to_centroid(point,current_point)
            points.append(point)
    
    scores = map(fun.evaluate, points)
    current_patience = 0
    best_sol, best_sol_value = current_point.copy(), fun.evaluate(current_point)
    while True:
        worst_point_index = np.argmax(scores)
        centroid = calculate_centroid(worst_point_index, points)
        centroid_score = fun.evaluate(centroid)
        
        if best_sol_value < centroid_score:
            current_patience +=1
            if(current_patience > patience):
                print "No improvement, stopping search with best found solution %s" %(best_sol)
                return centroid
        else:
            current_patience = 0
            best_sol, best_sol_value = centroid, centroid_score
        
        reflection = (1+alpha) * centroid - alpha*points[worst_point_index]
        for i in range(dimensions):
            if per_variable_restriction and i < len(explicit_restrictions):
                if reflection[i] < explicit_restrictions[i][0]:
                              reflection[i] = explicit_restrictions[i][0]
                elif reflection[i] > explicit_restrictions[i][1]:
                              reflection[i] = explicit_restrictions[i][1]
            else:
                if reflection[i] < explicit_restrictions[0]:
                              reflection[i] = explicit_restrictions[0]
                elif reflection[i] > explicit_restrictions[1]:
                              reflection[i] = explicit_restrictions[1]
                        
        while fun.check_inequality_constraints(reflection) == False:
                reflection = move_to_centroid(reflection,centroid)
                
        points[worst_point_index] = reflection
        scores[worst_point_index] = fun.evaluate(reflection)
        if worst_point_index == np.argmax(scores):
                reflection = move_to_centroid(reflection,centroid)
                points[worst_point_index] = reflection
                
        if stopping_condition(centroid,points, epsilon):
            return centroid

# Minimizacija problema bez ogranicenja

In [88]:
def min_no_restriction_func(fun,starting_point,epsilon=1e-6):
    
    current_point = np.array(starting_point,dtype=np.float32)
    fun.set_t(1.)
    while True:
        new_point = nelder_mead(fun,current_point)
        if (abs(new_point-current_point)<epsilon).all():
            print "Difference between new and old points less than %s, breaking the search" % (epsilon)
            return new_point
        current_point=new_point
        fun.set_t(fun.t*2)

# Zad 1

In [92]:
print "bez optimalnog koraka"
print grad_descent(F3(),[0.,0.], use_golden_section=False)
print
print "sa optimalnim korakom"
print grad_descent(F3(),[0.,0.],use_golden_section=True)

bez optimalnog koraka
No improvement, stopping search with best found solution [ 0.  0.]
[ 0.  0.]

sa optimalnim korakom
Euclidean norm 5.3312e-07 less than od 0.01 stopping search
[ 1.99999988 -2.99999976]


# Zad 2 - Rosenbrock

In [129]:
print "Rosenbrock"
print "Gradijentni spust"
print "sa optimalnim korakom"
fun = Rosenbrock()
print grad_descent(fun,[-1.9,2.],use_golden_section=True,grad_norm=5e-2)
print "evaluations = %s, gradients = %s" % (fun.calls, fun.gradient_calls)
print "_____________________________"
print "Newton rhapson"
print "sa optimalnim korakom"
fun = Rosenbrock()
print newton_rhapson(fun,[-1.9,2.],use_golden_section=True)
print "evaluations = %s, gradients = %s, hessean = %s" % (fun.calls, fun.gradient_calls,fun.hessean_calls)

Rosenbrock
Gradijentni spust
sa optimalnim korakom
Euclidean norm 0.0497096 less than od 0.05 stopping search
[ 0.96152251  0.9244483 ]
evaluations = 216094, gradients = 2004
_____________________________
Newton rhapson
sa optimalnim korakom
Euclidean norm 0.00461999 less than od 0.01 stopping search
[ 1.00122094  1.0036006 ]
evaluations = 1065, gradients = 14, hessean = 14


# Zad2 - F2

In [130]:
print "Funkcija 2"
print "Gradijentni spust"
print "sa optimalnim korakom"
fun = F2()
print grad_descent(fun,[0.1,0.3],use_golden_section=True)
print "evaluations = %s, gradients = %s" % (fun.calls, fun.gradient_calls)
print "_____________________________"
print "Newton rhapson"
print "sa optimalnim korakom"
fun = F2()
print newton_rhapson(fun,[0.1,0.3],use_golden_section=True)
print "evaluations = %s, gradients = %s, hessean = %s" % (fun.calls, fun.gradient_calls,fun.hessean_calls)

Funkcija 2
Gradijentni spust
sa optimalnim korakom
Euclidean norm 0.00416049 less than od 0.01 stopping search
[ 3.99819663  2.00025924]
evaluations = 1391, gradients = 15
_____________________________
Newton rhapson
sa optimalnim korakom
Euclidean norm 5.3312e-07 less than od 0.01 stopping search
[ 4.00000048  2.00000024]
evaluations = 119, gradients = 3, hessean = 3


# Zad 3 Box na Rosenbrock i F2

In [68]:
print "Rosenbrock"
rosen = BoxOptFunction(Rosenbrock(),[First_restriction(inequality_rest=False,power=1),Second_Restriction(inequality_rest=False,power=1)])
print box_opt(rosen,[-1.9,2],[-100,100])
print "Evaluations = ", rosen.calls
print
print "F2"
f2 = BoxOptFunction(F2(),[First_restriction(inequality_rest=False,power=1),Second_Restriction(inequality_rest=False,power=1)])
print box_opt(f2,[0.1,0.3],[-100,100])
print "Evaluations = ",f2.calls

Rosenbrock
No improvement, stopping search with best found solution [ 1.00543433  1.01073486]
[ 1.00587781  1.01185676]
Evaluations =  277

F2
[ 1.99999995  2.00021966]
Evaluations =  266


# Zad 4

In [89]:
print "Rosenbrock"
rosen = MixedTransformationFunction(Rosenbrock(),[First_restriction(inequality_rest=True,power=1),Second_Restriction(inequality_rest=True,power=1)])
print min_no_restriction_func(rosen,[-1.9,2])
print "Evaluations = ", rosen.calls
print
print "F2"
f2 = MixedTransformationFunction(F2(),[First_restriction(inequality_rest=True,power=1),Second_Restriction(inequality_rest=True,power=1)])
print min_no_restriction_func(f2,[0.1,0.3])
print "Evaluations = ",f2.calls

Rosenbrock
Difference between new and old points less than 1e-06, breaking the search
[ 0.0101129   0.01014262]
Evaluations =  2098

F2
Difference between new and old points less than 1e-06, breaking the search
[ 1.99998636  2.01660061]
Evaluations =  1978


# Zadatak 5 - bez unutarnje tocke

In [90]:
f4 = MixedTransformationFunction(F4(),[Third_Restriction(inequality_rest=True,power=1),Fourth_Restriction(inequality_rest=True,power=1)],[Fifth_Restriction(inequality_rest=False)])
print min_no_restriction_func(f4,[5.,5.])
print "Evaluations = ", f4.calls

Difference between new and old points less than 1e-06, breaking the search
[ 5.  5.]
Evaluations =  52


# Zadatak 5 - s unutarnjom tockom

In [142]:
finder = Inner_Point_Search([Third_Restriction(inequality_rest=False,power=1),Fourth_Restriction(inequality_rest=False,power=1)])
starting = finder.get_inner_point(2)
f4 = MixedTransformationFunction(F4(),[Third_Restriction(inequality_rest=True,power=1),Fourth_Restriction(inequality_rest=True,power=1)],[Fifth_Restriction(inequality_rest=False)])
print min_no_restriction_func(f4,starting)
print "Evaluations = ", f4.calls

Difference between new and old points less than 1e-06, breaking the search
[ 2.00043168  0.99950902]
Evaluations =  1943
